## Introduction to Pandas

[Pandas](http://pandas.pydata.org/) is the essential data analysis library for Python programmers. It provides fast and flexible data structures built on top of [numpy](http://www.numpy.org/).

It is well suited to handle "tabular" data (that might be found in a spreadsheet), time series data, or pretty much anything you care to put in a matrix with rows and named columns.

It contains two primary data structures, the `Series` (1-dimensional) and the `DataFrame` (2-dimensional) as well as a host of convenience methods for loading and plotting data.

The main thing that makes pandas pandas is that all data is *intrinsically aligned*. That means each data structure, `DataFrame` or `Series` has something called an **Index** that links data values with a label. That link will always be there (unless you explicitly break or change it) and it's what allows pandas to quickly and efficiently "do the right thing" when working with data.

In [1]:
# The canonical way to import pandas:
import pandas as pd
import numpy as np

## The `Series` Object

A `Series` is a one-dimensional array of indexed data.

In [2]:
data = pd.Series([0.1, 0.2, 0.3, 0.4])
data

0    0.1
1    0.2
2    0.3
3    0.4
dtype: float64

The `Series` wraps a 1-d `ndarray` from numpy and an `Index` object.

In [3]:
data.values

array([0.1, 0.2, 0.3, 0.4])

In [4]:
type(data.values)

numpy.ndarray

In [5]:
data.index

RangeIndex(start=0, stop=4, step=1)

In [6]:
# This particular index type, the `RangeIndex`, lets us use the
# same square-bracket notation as a `ndarray` to access elements:
data[0]

0.1

In [7]:
data.values[0]

0.1

In [8]:
# or even a slice:
data[1:3]

1    0.2
2    0.3
dtype: float64

We don't have to use this auto-generated list of integers as the index though. Index values can be specified manually and don't even have to be integers.

In [9]:
data = pd.Series([0.1, 0.2, 0.3, 0.4], index=['a', 'b', 'c', 'd'])
data

a    0.1
b    0.2
c    0.3
d    0.4
dtype: float64

In [10]:
data.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [11]:
# Item access works just like before, with square brackets, 
# even though the index values are strings
data['a']

0.1

In [12]:
# slices still work! But note the last element is included this time.
# This is the default behavior for indexes.
data['a':'c']

a    0.1
b    0.2
c    0.3
dtype: float64

In [13]:
# We could create a non-sequential integer index:
data = pd.Series([0.1, 0.2, 0.3, 0.4], index=[5, 8, 2, 1])
data

5    0.1
8    0.2
2    0.3
1    0.4
dtype: float64

In [14]:
data.index

Int64Index([5, 8, 2, 1], dtype='int64')

In [15]:
data.values[1]

0.2

In [16]:
# Why?
data[1]

0.4

Above we see the critical difference between numpy arrays, which are always ordered sequentially and have an implicit integer index, and `Series` objects, which have an index that maps *labels* to *values*.

`Series` are in fact a cross between a numpy array and a python dictionary. You can think of them as a dictionary with *typed* keys and *typed* values.

In [17]:
max_depths_dict = {
    'Erie': 64,
    'Huron': 229,
    'Michigan': 281,
    'Ontario': 244,
    'Superior': 406,
}
max_depths = pd.Series(max_depths_dict)
max_depths

Erie         64
Huron       229
Michigan    281
Ontario     244
Superior    406
dtype: int64

In [18]:
# squint and it looks like a dictionary!
max_depths['Michigan']

281

In [19]:
max_depths_dict['Michigan']

281

In [20]:
# Notice the index in this case was constructed automatically
# from the dictionary keys.
max_depths.index

Index(['Erie', 'Huron', 'Michigan', 'Ontario', 'Superior'], dtype='object')

We can think of an `Index` as an *immutable*, n-dimensional array. 

## `Series` Exercise

Create `Series` objects from a list, a numpy 1-dimensional `ndarray`, and a dictionary.

## Numpy and `Series`

Because the values in a `Series` are contained in a numpy `ndarray`, `Series` provides all the benefits of numpy! Namely, this means we get ultra-fast vectorized math operations on the elements of a `Series`.

In [21]:
max_depths * 10

Erie         640
Huron       2290
Michigan    2810
Ontario     2440
Superior    4060
dtype: int64

You can use most numpy functions directly on a `Series` object (and later, we'll see `DataFrame` objects as well), but pandas also provides access to these numpy functions through the `Series` object methods.

In [22]:
np.sin(max_depths)

Erie        0.920026
Huron       0.329962
Michigan   -0.985151
Ontario    -0.864536
Superior   -0.670252
dtype: float64

In [23]:
np.mean(max_depths)

244.8

In [24]:
max_depths.mean()

244.8

`Series` objects also support numpy-style Boolean mask indexing:

In [25]:
max_depths[max_depths > max_depths.mean()]

Michigan    281
Superior    406
dtype: int64

And numpy's so-called "fancy indexing", IE using a list or array to specify values to access:

In [26]:
max_depths[['Erie', 'Huron']]

Erie      64
Huron    229
dtype: int64

## The DataFrame Object

Much like the `Series` is a one-dimensional array of indexed data, a `DataFrame` is a two-dimensional array of indexed data.

You can think of a `DataFrame` as a sequence of `Series` objects all sharing the same index.

In [27]:
avg_depths_dict = {
    'Erie': 19,
    'Huron': 59,
    'Michigan': 85,
    'Ontario': 86,
    'Superior': 149,
}

avg_depths = pd.Series(avg_depths_dict)

lakes = pd.DataFrame({'Max Depth (m)': max_depths, 'Avg Depth (m)': avg_depths})
lakes

,Max Depth (m),Avg Depth (m)
Erie,64,19
Huron,229,59
Michigan,281,85
Ontario,244,86
Superior,406,149


Just like the `Series`, a `DataFrame` has an `index` property.

In [28]:
lakes.index

Index(['Erie', 'Huron', 'Michigan', 'Ontario', 'Superior'], dtype='object')

And a `values` property that exposes the underlying `ndarray`.

In [29]:
lakes.values

array([[ 64,  19],
       [229,  59],
       [281,  85],
       [244,  86],
       [406, 149]])

And unlike the Series, the DataFrame has a `columns` property, which is also an index.

In [30]:
lakes.columns

Index(['Max Depth (m)', 'Avg Depth (m)'], dtype='object')

We can get the shape of a dataframe, just like a numpy ndarray:

In [31]:
lakes.shape

(5, 2)

We can do dictionary-style lookups into the dataframe by column name to get a single `Series`:

In [32]:
lakes['Max Depth (m)']

Erie         64
Huron       229
Michigan    281
Ontario     244
Superior    406
Name: Max Depth (m), dtype: int64

To select more than one column put a list of column names inside the dictionary-style square brackets:

In [33]:
lakes[['Max Depth (m)','Avg Depth (m)']]

,Max Depth (m),Avg Depth (m)
Erie,64,19
Huron,229,59
Michigan,281,85
Ontario,244,86
Superior,406,149


### Creating new columns

Once we have a `DataFrame`, creating new columns is done through simple assignment.

In [34]:
surface_area = pd.Series({
    'Superior': 82097,
    'Michigan': 57753,
    'Huron': 59565,
    'Erie': 25655,
    'Ontario': 19009,
})

lakes['Surface Area (sq km)'] = surface_area
lakes

,Max Depth (m),Avg Depth (m),Surface Area (sq km)
Erie,64,19,25655
Huron,229,59,59565
Michigan,281,85,57753
Ontario,244,86,19009
Superior,406,149,82097


Notice how the index values allowed pandas to "align" the new data with the existing data!

It's also possible to create new columns from existing columns. Say for example we wanted a column to track the difference between the avg depth and max depth. We'll call this the "depth spread".

In [35]:
lakes['Depth Spread'] = lakes['Max Depth (m)'] - lakes['Avg Depth (m)']
lakes

,Max Depth (m),Avg Depth (m),Surface Area (sq km),Depth Spread
Erie,64,19,25655,45
Huron,229,59,59565,170
Michigan,281,85,57753,196
Ontario,244,86,19009,158
Superior,406,149,82097,257


DataFrames can be created from many different kinds of data structures (`Series` objects, lists, dictionaries, numpy arrays, etc.)

If you don't specify an index explicitly when creating the DataFrame, or you are using data without implicit indexes, pandas will create a `RangeIndex` for you:

In [36]:
call_signs = ['WLUW', 'WNUR', 'WBEZ', 'WXRT', 'WFMT']
frequencies = [88.7, 89.3, 91.5, 93.1, 98.7]
formats = ['College', 'College', 'Public Radio', 'Adult Album Alternative', 'Classical']
radio_station_df = pd.DataFrame(data={'Call Sign': call_signs, 'Frequency': frequencies, 'Format': formats})
radio_station_df

,Call Sign,Frequency,Format
0,WLUW,88.7,College
1,WNUR,89.3,College
2,WBEZ,91.5,Public Radio
3,WXRT,93.1,Adult Album Alternative
4,WFMT,98.7,Classical


### Setting the index

You may want to "move" one of the columns to be the index. You can do this with the DataFrame's `set_index` method. By default this returns a new DataFrame with the index replaced with the values in the chosen column.

The `inplace` parameter will make the change to the existing DataFrame rather than returning a new one.

In [37]:
radio_station_df.set_index('Call Sign')

,Frequency,Format
Call Sign,,
WLUW,88.7,College
WNUR,89.3,College
WBEZ,91.5,Public Radio
WXRT,93.1,Adult Album Alternative
WFMT,98.7,Classical


It is possible to move the index back to a column with the `reset_index` method:

In [38]:
radio_station_df.reset_index()
radio_station_df

,Call Sign,Frequency,Format
0,WLUW,88.7,College
1,WNUR,89.3,College
2,WBEZ,91.5,Public Radio
3,WXRT,93.1,Adult Album Alternative
4,WFMT,98.7,Classical


## DataFrame Exercise

Create a DataFrame using the data in these lists:
```
titles = ["Dangerously in Love", "B'Day", "I Am... Sasha Fierce", "4", "Beyoncé", "Lemonade"]
release_dates = ["2003-06-23", "2006-09-01", "2008-11-14", "2011-06-24", "2013-12-13", "2016-04-23"]
total_sales = [11000000, 8000000, 8000000, 2600000, 5000000, 2500000]
us_sales = [5000000, 3410000, 3200000, 1500000, 2300000, 1554000]
```

Each list should be a column.

Now change the DataFrame so that the index values are the album titles.

Create a new column, "Non-US Sales", that contains the difference of the Total Sales and US Sales columns.

## Data Indexing and Selection

Now that we can load data into pandas objects, we need to be able to access it. Pandas offers a variety of methods for accessing the data we need.

First, both `Series` and `DataFrame` objects support dictionary-style access with square brackets. Think of index label values as dictionary keys:

In [39]:
# We saw this above -- access a series like a dictionary to get a single value.
avg_depths['Michigan']

85

In [40]:
# DataFrame dictionary-style access returns the Series with that column index label:
lakes['Avg Depth (m)']

Erie         19
Huron        59
Michigan     85
Ontario      86
Superior    149
Name: Avg Depth (m), dtype: int64

Boolean masking and fancy indexing work with DataFrames, just like Series objects:

In [41]:
# use a Boolean mask to select just the items we want:
lakes[avg_depths > 60]

,Max Depth (m),Avg Depth (m),Surface Area (sq km),Depth Spread
Michigan,281,85,57753,196
Ontario,244,86,19009,158
Superior,406,149,82097,257


This works because the Boolean mask creates a new `Series` with the same index values!

In [42]:
avg_depths > 60

Erie        False
Huron       False
Michigan     True
Ontario      True
Superior     True
dtype: bool

To select a subset of columns from a DataFrame, use fancy indexing with the column names in a list:

In [43]:
lakes[['Avg Depth (m)', 'Max Depth (m)']]

,Avg Depth (m),Max Depth (m)
Erie,19,64
Huron,59,229
Michigan,85,281
Ontario,86,244
Superior,149,406


In [44]:
# There is a potential problem with non-sequential integer indexes:
data_implicit = pd.Series([100, 200, 300, 400])
data_explicit = pd.Series([100, 200, 300, 400], index=[4, 9, 8, 1])
print(data_implicit)
print()
print(data_explicit)

0    100
1    200
2    300
3    400
dtype: int64

4    100
9    200
8    300
1    400
dtype: int64


To handle this potential confusion between label-based and position-based access and make data access easier in general, pandas provides three "indexers": `Series` and `DataFrame` attributes that expose differents ways to access the data.

- `iloc`: always integer position-based
- `loc`: always label-based


- `ix`: primarily label-based, falls back to position-based.

In [45]:
data_implicit.iloc[1]

200

In [46]:
data_explicit.iloc[1]

200

In [47]:
data_implicit.loc[4]  # Note that this should result in an error

KeyError: 'the label [4] is not in the [index]'

In [ ]:
data_explicit.loc[4] # # Note that this does not result in an error

In [48]:
# We can use slices to select more than one value as well. Here, get all values after the first one:
data_implicit.iloc[1:]

1    200
2    300
3    400
dtype: int64

In [49]:
# ix is useful with DataFrames and allows you to mix label and position-based
lakes.ix[0, ['Avg Depth (m)', 'Max Depth (m)']]

/Users/tstoeger/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


Avg Depth (m)    19
Max Depth (m)    64
Name: Erie, dtype: int64

In [ ]:
# Pop quiz! Let's get all rows of the lakes dataframe except the last one:
lakes.iloc[0:-1]

In [ ]:
# What about the first two rows and first two columns only?
lakes.iloc[:2, :2]

In [ ]:
lakes.loc['Erie']

`loc` accepts the following types of inputs:

- a single label (as above)
- a list or array of labels, e.g. ['a', 'b', 'c']
- a slice object with labels e.g. 'a':'c' (note that contrary to usual python slices, both the start and the stop are included!)
- A boolean array
- A callable function with one argument (the calling Series, DataFrame or Panel) and that returns valid output for indexing (one of the above)

`loc` and `iloc` also take an optional second parameter, the list of column names to return:

In [ ]:
lakes.loc[['Michigan', 'Superior'], ['Max Depth (m)']]

It is also possible to assign to the values at the locations you specify with the `iloc`, `loc`, or `ix` indexers! They aren't read-only.

In [ ]:
df = pd.DataFrame(np.random.randint(0, 10, (3, 3)), columns=['A', 'B', 'C'])
df

In [ ]:
# Assign the value 100 to the cells 0,B and 1,B.
# Remember with label-based access, which `loc` uses, the "stop" of the slice is *included*.
df.loc[:1, 'B'] = 100
df

## DataFrame Selection Exercise

Construct a DataFrame from the following data (note you can use the `index` parameter to `pd.DataFrame` to set the index when creating a DataFrame):
```
data = {'animal': ['cat', 'cat', 'snake', 'dog', 'dog', 'cat', 'snake', 'cat', 'dog', 'dog'],
        'age': [2.5, 3, 0.5, np.nan, 5, 2, 4.5, np.nan, 7, 3],
        'visits': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
        'priority': ['yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no']}

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
```

Display the first three rows using the `.iloc` indexer:

Display the last three rows using the `.iloc` indexer:

Use the `.loc` indexer to access rows `a` through `d`:

Display just the "animal" and "age" columns (you can use either the `.loc` indexer or dictionary-style access for this):

Select the data where the animal's age is greater than 3:

### Examining Data

While you can manipulate and operate on your data in any way you can dream up, pandas does provide basic descriptive statistics and sorting functionality for you. I **highly** recommend reading the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/api.html#computations-descriptive-stats) to see what methods are available and save yourself some work!

The `describe` method is very useful with numeric data:

In [ ]:
lakes.describe()

We can get the highest value for a given Series with `max`:

In [ ]:
max_depths.max()

But what if we wanted the top 2? `sort_values` is the answer:

In [ ]:
max_depths.sort_values(ascending=False).head(2)

This is so common that there is actually a shortcut for it:

In [ ]:
max_depths.nlargest(2)

Which naturally works on DataFrames as well:

In [ ]:
lakes.nlargest(2, 'Avg Depth (m)')

## Loading Data

Pandas provides a bunch of functions for reading data from a variety of sources, including CSV, Excel files, SQL databases, HDF5, even your computer's clipboard! The always-comprehensive pandas documentation has more info here: [https://pandas.pydata.org/pandas-docs/stable/io.html](https://pandas.pydata.org/pandas-docs/stable/io.html).

Let's read a local CSV dataset into a dataframe using the `read_csv` function.

In [ ]:
df = pd.read_csv("data/Speed_Camera_Violations.csv")

This particular `DataFrame` contains speed camera violation data provided by the city of Chicago. This dataset is available at [https://catalog.data.gov/dataset/speed-camera-violations-997eb](https://catalog.data.gov/dataset/speed-camera-violations-997eb).

Let's start inspecting it by using the `head` method to look at the first five rows.

In [ ]:
df.head(10)

When data is loaded from an external source, pandas will try to guess the datatype for each column. Let's see how it did:

In [ ]:
pd.Series({col: df[col].dtype for col in df.columns})

## Data types

Much of pandas functionality depends on the data types of the `Series` it's working with. For instance we can get summary measures and do numpy-like parallel operations on numeric types (`int64`, `float64`), or do date-based arithmetic with `date` series.

Notice above that the data type of the `VIOLATION DATE` column is "object", which, just like in numpy, means it is a generic type that isn't very useful. Let's turn those date strings into actual date objects, which are much better to work with.

In [ ]:
# given a Series, pd.to_datetime returns a new Series with the string dates parsed as actual dates.
# We can then directly assign that Series back to the original column in our dataframe and pandas' magical Index
# functionality will make it all line up properly.
df["VIOLATION DATE"] = pd.to_datetime(df["VIOLATION DATE"], format="%m/%d/%Y")

df["VIOLATION DATE"].head()

## Filtering

Now that we have a date column, we can do things like filter to only look at violations in 2015.

To do this, we'll create a "filter", essentially a boolean expression that works just like a mask or "fancy indexing" expression in numpy, and apply that filter to our dataframe to get just the rows we want.


In [ ]:
import datetime

# note the extra parentheses below, these are necessary when creating a boolean filter expression with
# multiple comparisons like this
date_filter = ((df["VIOLATION DATE"] >= datetime.date(2015,1,1)) & (df["VIOLATION DATE"] < datetime.date(2016,1,1)))

# date_filter now contains a series of true/false values that we can use to extract just the values we are interested in
# by putting it in square brackets after the dataframe variable.
print(date_filter.head())
print()
print(date_filter.tail())

df_2015 = df[date_filter]

df_2015.head()

This kind of filtering works for any kind of data type, provided you take care to make sure pandas is using the right data types for your data!

You may have noticed that many of the rows in this dataframe are missing lat/lon data. Pandas uses the "NaN" placeholder for missing data and offers some methods for dealing with it.

Both `Series` and `DataFrame` objects have `fillna` method that will replace missing data with a specified value.

In thise case however we may want to just drop those records that have missing data entirely:


In [ ]:
df_no_nans = df.dropna(axis=0, how="any")
df_no_nans.head()